In [1]:
#Importing Packages
import csv
import pandas as pd
import numpy as np
import datetime
from datetime import timedelta

In [2]:
# Reading in Data (population data is from US Census, hard coded)
url = ('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-states.csv')
df0 = pd.read_csv(url)
raw_data = {'State': ['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 'Delaware', 'District of Columbia', 'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming', 'Puerto Rico'], 'Estimated_Pop': [4903185, 731545, 7278717, 3017804, 39512223, 5758736, 3565287, 973764, 705749, 21477737, 10617423, 1415872, 1787065, 12671821, 6732219, 3155070, 2913314, 4467673, 4648794, 1344212, 6045680, 6892503, 9986857, 5639632, 2976149, 6137428, 1068778, 1934408, 3080156, 1359711, 8882190, 2096829, 19453561, 10488084, 762062, 11689100, 3956971, 4217737, 12801989, 1059361, 5148714, 884659, 6829174, 28995881, 3205958, 623989, 8535519, 7614893, 1792147, 5822434, 578759, 3193694]}
pops = pd.DataFrame(raw_data, columns = ['State', 'Estimated_Pop'])

In [3]:
# Calculating Population in Millions
pops["Population_millions"] = pops["Estimated_Pop"]/1000000

# filter data to only most recent date in data
df0["last_updated"] = max(df0["date"])
df1 = df0[(df0['date'] == df0['last_updated'])]

# join to population data and calculate per capitas as well as death rate
df3 = df1.merge(pops, how='inner', right_on="State", left_on="state")
df3["cases_per_hundred_thousand"] = df3["cases"]/df3["Population_millions"]/10
df3["deaths_per_hundred_thousand"] = df3["deaths"]/df3["Population_millions"]/10
df3["deaths_per_case"] = df3["deaths"]/df3["cases"]

In [4]:
# Convoluted logic for creating minimum and maximum dates, then filtering data based on it
delta = datetime.timedelta(days= 3)
from datetime import datetime
max_date = datetime.strptime((max(df0["date"])), '%Y-%m-%d')
min_date = max_date - delta
df0['date'] = pd.to_datetime(df0['date'], format = '%Y-%m-%d')
df1 = df0[(df0['date']> min_date)] 

In [5]:
# Creating three day case growth rate and adding to general df
three_day_case_growth = df1.groupby('state')['cases'].agg(np.ptp)/ df1.groupby('state')['cases'].min()
growth_df =pd.DataFrame({'state':three_day_case_growth.index, 'three_day_case_growth':three_day_case_growth.values})
df4 = df3.merge(growth_df, how='left', right_on="state", left_on="state")
final_df = df4.drop(['State', 'date', 'fips'], axis=1)
final_df

,state,cases,deaths,last_updated,Estimated_Pop,Population_millions,cases_per_hundred_thousand,deaths_per_hundred_thousand,deaths_per_case,three_day_case_growth
0,Alabama,999,13,2020-03-31,4903185,4.903185,20.374512,0.265134,0.013013,0.203614
1,Alaska,133,2,2020-03-31,731545,0.731545,18.180700,0.273394,0.015038,0.166667
2,Arizona,1298,24,2020-03-31,7278717,7.278717,17.832813,0.329728,0.018490,0.397201
3,Arkansas,564,8,2020-03-31,3017804,3.017804,18.689087,0.265093,0.014184,0.256125
4,California,8582,183,2020-03-31,39512223,39.512223,21.719861,0.463148,0.021324,0.369614
5,Colorado,2990,69,2020-03-31,5758736,5.758736,51.921116,1.198180,0.023077,0.291577
6,Connecticut,3128,69,2020-03-31,3565287,3.565287,87.734872,1.935328,0.022059,0.569493
7,Delaware,319,10,2020-03-31,973764,0.973764,32.759478,1.026943,0.031348,0.375000
8,District of Columbia,495,9,2020-03-31,705749,0.705749,70.138250,1.275241,0.018182,0.447368
9,Florida,6741,85,2020-03-31,21477737,21.477737,31.385988,0.395759,0.012609,0.364023


In [6]:
final_df.to_csv("Covid-19 Case and death data by state.csv", encoding = 'utf-8')